In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
#scene_dataset = "C:\\Users\\MatyuGuerrero\\Desktop\\Python\\Senior Project\\scenes" what data set is doing
scene_dataset = 'scenes' #put the filepath for the dataset
dataset_path = os.listdir(scene_dataset)
scene_types = os.listdir(scene_dataset)
print("The Scene Types are: ", scene_types) #what kind of scnene images are in this data set
print("Scenes types found:", len(scene_types))

The Scene Types are:  ['indoorCVPR_09_imgs', 'outdoorCVPR_09_imgs']
Scenes types found: 2


In [2]:
scenes = []

for item in scene_types: #This first for loop enters in our folder
    all_scenes = os.listdir(scene_dataset + '/' + item) #Gets all the file names
    #print(all_scenes)

    for scene in all_scenes: #This second list will get all the images of our folder
        scenes.append((item, str(scene_dataset + '/' + item) + '/' + scene)) 
        #print(scenes[:1])
scenes

[('indoorCVPR_09_imgs', 'scenes/indoorCVPR_09_imgs/00.jpg'),
 ('indoorCVPR_09_imgs', 'scenes/indoorCVPR_09_imgs/000000.jpg'),
 ('indoorCVPR_09_imgs', 'scenes/indoorCVPR_09_imgs/000106949b.jpg'),
 ('indoorCVPR_09_imgs', 'scenes/indoorCVPR_09_imgs/000106949d.jpg'),
 ('indoorCVPR_09_imgs', 'scenes/indoorCVPR_09_imgs/00036_NiagaraFalls4.jpg'),
 ('indoorCVPR_09_imgs', 'scenes/indoorCVPR_09_imgs/00040220.jpg'),
 ('indoorCVPR_09_imgs', 'scenes/indoorCVPR_09_imgs/0006.jpg'),
 ('indoorCVPR_09_imgs', 'scenes/indoorCVPR_09_imgs/0009.jpg'),
 ('indoorCVPR_09_imgs', 'scenes/indoorCVPR_09_imgs/000_0141_0001.jpg'),
 ('indoorCVPR_09_imgs',
  'scenes/indoorCVPR_09_imgs/00114320c9df07c3329506.jpg'),
 ('indoorCVPR_09_imgs', 'scenes/indoorCVPR_09_imgs/0012.jpg'),
 ('indoorCVPR_09_imgs', 'scenes/indoorCVPR_09_imgs/0020.jpg'),
 ('indoorCVPR_09_imgs', 'scenes/indoorCVPR_09_imgs/0021550.jpg'),
 ('indoorCVPR_09_imgs', 'scenes/indoorCVPR_09_imgs/003.jpg'),
 ('indoorCVPR_09_imgs', 'scenes/indoorCVPR_09_imgs/00320

In [3]:

scenes_dataframe = pd.DataFrame(data = scenes, columns = ['scene type', 'image']) #scene type shows the class label
print(scenes_dataframe.head())
#print(scenes_dataframe.tail())

           scene type                                              image
0  indoorCVPR_09_imgs                   scenes/indoorCVPR_09_imgs/00.jpg
1  indoorCVPR_09_imgs               scenes/indoorCVPR_09_imgs/000000.jpg
2  indoorCVPR_09_imgs           scenes/indoorCVPR_09_imgs/000106949b.jpg
3  indoorCVPR_09_imgs           scenes/indoorCVPR_09_imgs/000106949d.jpg
4  indoorCVPR_09_imgs  scenes/indoorCVPR_09_imgs/00036_NiagaraFalls4.jpg


In [4]:
print("Total number of scenes in the dataset:", len(scenes_dataframe))

scene_count = scenes_dataframe['scene type'].value_counts()

print("scnenes in each category:")
print(scene_count)

Total number of scenes in the dataset: 9936
scnenes in each category:
scene type
outdoorCVPR_09_imgs    5000
indoorCVPR_09_imgs     4936
Name: count, dtype: int64


In [5]:
import cv2

new_path = 'scenes/'

img_size = 224

images = []
labels = []

for i in scene_types:
    data_path =  new_path + str(i) #uhhhhhhhhh it works?
    filenames = [i for i in os.listdir(data_path)]
    #print(filenames) #get the names of all the imgs

    for f in filenames:
        img = cv2.imread(data_path + '/' + f)
        
        #catches 'bad' imgs 
        if img is None:
            print(f"Error loading jpg: {data_path}/{f}")
            continue
        
        #print(img) #will get img as an array
        img = cv2.resize(img, (img_size, img_size))
        images.append(img)
        labels.append(i)

labels

KeyboardInterrupt: 

In [ ]:
#transform the image to a numpy type

images = np.array(images)
images.shape 
#prints out (total number of imgs, the length size, the width size, the number of channels (rgb thus 3))

(9936, 224, 224, 3)

In [ ]:
images = images.astype('float32')/255.0 #divides the pixel number just to make it smaller 
#not really needed but its good for simplicity 
#Uncomment and test after reduce the img list size

In [ ]:
from sklearn.preprocessing import LabelEncoder , OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

y = scenes_dataframe['scene type'].values
#print(y[:5])

#for y 
y_labelencoder = LabelEncoder ()
y_encoded = y_labelencoder.fit_transform (y)
#print(y)
#print(y_encoded)


y = y.reshape(-1, 1)

ct = ColumnTransformer([('OneHotEncoder', OneHotEncoder(), [0])], remainder = 'passthrough')

Y = ct.fit_transform(y)
Y.shape
#print(Y)



(9936, 2)

In [ ]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

images, Y = shuffle(images, Y, random_state = 1)

train_x, test_x, train_y, test_y = train_test_split(images, Y, test_size = 0.05, random_state = 415)

#impact the shape of the training and testing
print(train_x.shape) #(number of images)
print(train_y.shape)
print(test_x.shape) #(number to test)
print(test_y.shape)

(9439, 224, 224, 3)
(9439, 2)
(497, 224, 224, 3)
(497, 2)


In [ ]:
import numpy
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, DepthwiseConv2D, GlobalAvgPool2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from tensorflow.python.keras.utils import layer_utils
from tensorflow.python.keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input

import pydot 
from IPython.display import SVG
from tensorflow.python.keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.initializers import glorot_uniform

import scipy.misc
from matplotlib.pyplot import imshow
from keras.initializers import glorot_uniform
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D


In [ ]:
def mobilenet_block(x, f, s = 1):
    x = DepthwiseConv2D(3, strides = s, padding = 'same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(f, 1)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    return x

In [ ]:
def mobilenet(input_shape = (224, 224, 3), classes = 2):

    X = Input(input_shape)

    x = Conv2D(32, 3, strides = 2, padding ='same')(X)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = mobilenet_block(x, 64)

    x = mobilenet_block(x, 128, 2)
    x = mobilenet_block(x, 128)

    x = mobilenet_block(x, 256, 2)
    x = mobilenet_block(x, 256)

    x = mobilenet_block(x, 512, 2)
    for _ in range(5):
        x = mobilenet_block(x, 512)

    x = mobilenet_block(x, 1024, 2)
    x = mobilenet_block(x, 1024)

    x = GlobalAvgPool2D()(x)

    output = Dense(classes, activation = 'softmax')(x)

    model = Model(X, output)
    return model

In [ ]:
model = mobilenet(input_shape = (224, 224, 3), classes = 2)

In [ ]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv2d_28 (Conv2D)          (None, 112, 112, 32)      896       
                                                                 
 batch_normalization_54 (Ba  (None, 112, 112, 32)      128       
 tchNormalization)                                               
                                                                 
 activation_54 (Activation)  (None, 112, 112, 32)      0         
                                                                 
 depthwise_conv2d_26 (Depth  (None, 112, 112, 32)      320       
 wiseConv2D)                                                     
                                                                 
 batch_normalization_55 (Ba  (None, 112, 112, 32)      128 

In [ ]:
training_history = model.fit(train_x, train_y, epochs = 3, batch_size = 32)

Epoch 1/3




In [ ]:
preds = model.evaluate(test_x, test_y)
print("Loss = " + str(preds[0]))
print("Test Accuracy = " + str(preds[1]))

In [ ]:
fig, ax = plt.subplots(2,1)
fig.suptitle('Learning Curves')

ax[0].plot(training_history.history['loss'], color ='b', label ="Training loss")
ax[0].plot(training_history.history['val_loss'], color ='r', label = "validation loss",axes = ax[0])
legend = ax[0].legend(loc ='best', shadow = True)
ax[0].set_xlabel('Epochs')
ax[0].set_ylabel('Loss')


ax[1].plot(training_history.history['accuracy'], color ='b', label ="Training accuracy")
ax[1].plot(training_history.history['val_accuracy'], color ='r', label ="Validation accuracy")
legend = ax[1].legend(loc ='best', shadow = True)
ax[1].set_xlabel('Epochs')
ax[1].set_ylabel('Accuracy')

plt.tight_layout()

In [ ]:
from matplotlib.pyplot import imread

threshold = 0.5

#Uncomment out after

img_path =  'out3.jpg'
img_test = image.load_img(img_path, target_size = (224, 224))
x = image.img_to_array(img_test)
x = np.expand_dims(x, axis = 0)
x = preprocess_input(x)

print('Input image shape: ', x.shape)
my_image = imread(img_path)
imshow(my_image)

prediction = model.predict(x)